In [1]:
import numpy as np
# from scipy import random
from scipy.stats import multivariate_normal

D:\anaconda3\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [29]:
##################### EM algorithm #####################################
def L2_dist(x1, x2):
    return np.sqrt(np.sum((x1-x2)**2))

class EM_GMM():
    def __init__(self, K=3, dim=2, init_mu=None, init_sigma=None, init_phi=None,  max_iters=100):
        self.K = K
        self.dim = dim
        
        if(init_mu is None):
            init_mu = np.random.rand(3, 2)*10-5 ### Comment: make it large!!!!
        self.mu = init_mu
        if(init_sigma is None):
            init_sigma = np.zeros((K, dim, dim))
            for i in range(K):
                init_sigma[i] = np.eye(dim)
        self.sigma = init_sigma
        if(init_phi is None):
            init_phi = np.ones(self.K)/self.K
        self.phi = init_phi
        
        self.max_iter = max_iters
        
        
    def predict(self, X):
        self.X = X
        self.n_samples, self.n_features = X.shape
        
        self.init_em(self.X)

        log_likelihood = []
        for _ in range(self.max_iter):
            # E-step
            self.e_step()
            # M-step
            self.m_step()
            # Computing log-likelihood
            log_likelihood.append(self.log_likelihood(X))
        
        centroids = self.mu
        clusters = self._create_clusters(centroids)

        # classify samples as the index of their clusters
        return centroids, self._get_cluster_labels(clusters)
               
        
    def _create_clusters(self, centroids):
        # assign the samples to the closest centroids
        clusters = [[] for _ in range(self.K)]
        for idx, sample in enumerate(self.X):
            centroid_idx = self._closest_centroid(sample, centroids)
            clusters[centroid_idx].append(idx)
        return clusters
    
    def _get_cluster_labels(self, clusters):
        # each sample will get the label of the cluster it was assigned to
        labels = np.empty(self.n_samples)
        for cluster_idx, cluster in enumerate(clusters):
            for sample_idx in cluster:
                labels[sample_idx] = cluster_idx

        return labels
    
    def _closest_centroid(self, sample, centroids):
        # distance of the current sample to each centroid
        distances = [L2_dist(sample, point) for point in centroids]
        closest_idx = np.argmin(distances)
        return closest_idx


        
    def init_em(self, X):
        self.data = X
        self.num_points = X.shape[0]
        self.z = np.zeros((self.num_points, self.K))
    
    def e_step(self):
        for i in range(self.K):
            self.z[:, i] = self.phi[i] * multivariate_normal.pdf(self.data, mean=self.mu[i], cov=self.sigma[i])
        self.z /= self.z.sum(axis=1, keepdims=True)
    
    def m_step(self):
        sum_z = self.z.sum(axis=0)
        self.phi = sum_z / self.num_points
        self.mu = np.matmul(self.z.T, self.data)
        self.mu /= sum_z[:, None]
        for i in range(self.K):
            j = np.expand_dims(self.data, axis=1) - self.mu[i]
            s = np.matmul(j.transpose([0, 2, 1]), j)
            self.sigma[i] = np.matmul(s.transpose(1, 2, 0), self.z[:, i] )
            self.sigma[i] /= sum_z[i] 
            
    def log_likelihood(self, X):
        ll = []
        for d in X:
            tot = 0
            for i in range(self.K):
                tot += self.phi[i] * multivariate_normal.pdf(d, mean=self.mu[i], cov=self.sigma[i])
            ll.append(np.log(tot))
        return np.sum(ll)